## Import Packages

In [ ]:
import numpy       as np
import pandas      as pd
import matplotlib.pyplot as plt
import pywt

## Load Data

In [ ]:
# Load data with column names
path = 'https://github.com/ljwg3000/UNT_MEEN/blob/main/AI_tutorial/Week2/ExampleData?raw=true'
Data = pd.read_csv(path, sep=',',names=['time(s)', 'Acceleration(g)', 'Voltage(V)', 'Current(kA)'])
Data

Confirm each signal by plotting

In [ ]:
plt.figure(figsize=(12,8))

plt.subplot(3,1,1) # Acceleration signal
plt.plot(Data.iloc[:,0] , Data.iloc[:,1], color='r')
plt.ylabel('Acceleration (g)', fontsize=12, color='r')
plt.grid()

plt.subplot(3,1,2) # Voltage signal
plt.plot(Data.iloc[:,0] , Data.iloc[:,2], color='g')
plt.ylabel('Voltage (V)', fontsize=12, color='g')
plt.grid()

plt.subplot(3,1,3) # Current signal
plt.plot(Data.iloc[:,0] , Data.iloc[:,3], color=[0,0,1])
plt.ylabel('Current (kA)',fontsize=12, color='b')
plt.xlabel('time (s)', fontsize=12)
plt.grid()

plt.show()

.

.

## **Wavelet Transform (WT)-based Signal Decomposition**

### 🔎 Quick Refresher:

- **Why WT?**  
  Unlike FFT (global frequency content) or STFT (fixed time-frequency window),  
  the Wavelet Transform provides **multi-resolution analysis**:  
  - High frequencies → short time windows (good time resolution)  
  - Low frequencies → long time windows (good frequency resolution)  
- This makes WT very powerful for **non-stationary signals**.

---

### (1) Parameter setting for WT

#### **Key Parameters of WT**

- **Mother wavelet**  
  - The prototype wavelet used to generate all scaled/shifted versions.  
  - Examples: `'haar'`, `'db4'`, `'sym5'`, `'coif5'`, `'mexh'` (Mexican hat), `'morl'` (Morlet).  
  - Choice depends on the signal:  
    * `'haar'` → simple step-like signals  
    * `'dbN'` (Daubechies) → smooth engineering signals  
    * `'morl'` → time-frequency analysis with oscillatory nature  
  - To better understand, please find: https://www.mathworks.com/help/wavelet/gs/introduction-to-the-wavelet-families.html
  - Documentation of `PyWt`: https://pywavelets.readthedocs.io/en/latest/ref/wavelets.html

- **Level (decomposition depth)**  
  - How many times the signal is recursively split into approximation/detail parts.  
  - Higher level → captures lower frequency bands.  
  - Limited by signal length (`max_level = log2(N)` approximately).


In [ ]:
MotherWavelet = pywt.Wavelet('db4')   # Mother wavelet
Level         = 8                      # Wavelet levels

---

### (2) Wavelet Decomposition Implementation

#### **Coefficients from Decomposition**

- `pywt.wavedec()` returns a list of coefficient arrays:  
  * Approximation (`a`) → low-frequency content  
  * Details (`d`) → high-frequency content at each level  

In [ ]:
Data_Target = Data.iloc[:,2] # Select one sensor signal
Coefficient = pywt.wavedec(Data_Target, MotherWavelet, level=Level, axis=0)
len(Coefficient)

In [ ]:
# Confirm extracted coefficients as DataFrame
Coefficient_df = pd.DataFrame(Coefficient)
Coefficient_df

Confirm the sizes of each coefficient
* Time resolution gets higher for observing high frequency signals

In [ ]:
print('Size of a',Level, ' = ', len(Coefficient[0]))  # First coefficient : the lowest freqeuncy ragne

for i in range(1,Level+1):
    print('Size of d',Level+1-i , ' = ', len(Coefficient[i]))

- **Why different sizes?**  
  - Each decomposition step downsamples the signal by 2.  
  - So, higher-frequency detail signals have **finer time resolution** (more samples).  
  - Lower-frequency approximation signals have **fewer samples**.  

👉 This explains why in the code, `len(Coefficient[i])` decreases as the level increases.

.

---

### (3) Plotting of Wavelet Decompositiobn Result


- The raw signal is shown at the top.  
- Below, each subplot shows one coefficient (low-pass approximation or high-pass detail).  
- X-axis is adjusted to match the time duration of each coefficient.  
- Together, the plots illustrate how the signal energy is distributed across different frequency ranges.  


In [ ]:
plt.figure(figsize=(6,10))

plt.subplot(Level+2,1,1)    # Raw signal
plt.plot(Data.iloc[:,0], Data_Target, color='r')
plt.ylabel('Raw data')
plt.grid(alpha=0.3)

plt.subplot(Level+2,1,2)    # The lowest frequency range
Time_temp = np.arange( 0 , 0.2167 + 0.2167/(len(Coefficient[0])-1) , 0.2167/(len(Coefficient[0])-1) )
plt.plot(Time_temp , Coefficient[0])
plt.ylabel('a %d' %(Level))
plt.grid(alpha=0.3)

for k in range(1,Level+1):
    plt.subplot(Level+2,1,k+2)
    Time_temp = np.arange( 0 , 0.2167 + 0.2167/(len(Coefficient[k])-1) , 0.2167/(len(Coefficient[k])-1) )
    plt.plot(Time_temp , Coefficient[k])
    plt.ylabel('d %d'%(Level+1-k))
    plt.grid(alpha=0.3)

plt.tight_layout()
plt.show()

.

.

---

### (4) Wavelet-based Filtering: Low-pass and High-pass

- **Why filtering with Wavelets?**  
  Wavelet Transform decomposes a signal into different frequency bands (approximation = low-frequency, details = high-frequency).  
  By selectively **zeroing out some coefficients** and reconstructing the signal, we can build **filters** in the time-frequency domain.  



### ⚡ How it works

1. **Full decomposition**  
   - Use `pywt.wavedec()` to split the signal into approximation (`a`) and detail (`d`) coefficients.  
   - Each coefficient represents a specific frequency range.  

2. **Full reconstruction**  
   - Recombine all coefficients with `pywt.waverec()` → original signal is recovered (baseline check).  

3. **Low-pass filtering**  
   - Remove (set to zero) the **high-frequency detail coefficients**.  
   - Reconstruct → smoothed version of the signal (low-frequency content preserved).  

4. **High-pass filtering**  
   - Remove (set to zero) the **low-frequency approximation coefficients**.  
   - Reconstruct → highlights rapid changes or high-frequency features.  



In [ ]:
Data_Target = Data.iloc[:,1] # Select one sensor signal

# 1. Wavelet decomposition
coeffs = pywt.wavedec(Data_Target, 'db4', level=5)

# 2. Full reconstruction (baseline check)
x_rec = pywt.waverec(coeffs, 'db4')

# 3. Filtering example: remove high-frequency details
coeffs_lowpass = coeffs.copy()
for i in range(1, len(coeffs_lowpass)):   # Select the number of components to be filterd out
    coeffs_lowpass[i] = np.zeros_like(coeffs_lowpass[i])
x_lowpass = pywt.waverec(coeffs_lowpass, 'db4')

pd.DataFrame(coeffs_lowpass)

In [ ]:
# 4. Filtering example: remove low-frequency approximation
coeffs_highpass = coeffs.copy()
for i in range(5):   # Select the number of components to be filterd out
    coeffs_highpass[i] = np.zeros_like(coeffs_highpass[i])
x_highpass = pywt.waverec(coeffs_highpass, 'db4')

pd.DataFrame(coeffs_highpass)

In [ ]:
# 5. Plot comparison
plt.figure(figsize=(12,6))
plt.plot(Data_Target, label='Original', alpha=0.8)
plt.plot(x_rec, label='Reconstructed (all coeffs)', linestyle='--')
plt.plot(x_lowpass, label='Low-pass filtered (high-freq removed)')
plt.plot(x_highpass, label='High-pass filtered (low-freq removed)')
plt.legend()
plt.title("Wavelet Reconstruction and Filtering")
plt.grid(True)
plt.show()


### 📊 Interpretation

- **Low-pass filtered signal**  
  - Removes high-frequency noise.  
  - Keeps slow-changing baseline behavior.  
  - Useful for trend extraction and denoising.  

- **High-pass filtered signal**  
  - Removes baseline trend.  
  - Keeps oscillations, spikes, or anomalies.  
  - Useful for fault detection or vibration analysis.  

---

👉 With this approach, Wavelet filtering acts as a **flexible tool** to isolate frequency ranges in real signals, combining both **time** and **frequency** information.